In [1]:
%matplotlib inline
import utils; reload(utils)
from utils import *

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
# batch_size = 4
batch_size = 64

In [3]:
path = '../data/mnist/'
model_path = path + 'models/'

In [4]:
from keras.datasets import mnist

In [5]:
(X_train,y_train),(X_test,y_test) = mnist.load_data()

In [6]:
(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [7]:
X_train = np.expand_dims(X_train,1)
X_test = np.expand_dims(X_test,1)

In [8]:
X_train.shape,X_train.shape

((60000, 1, 28, 28), (60000, 1, 28, 28))

In [9]:
y_train[:5]

array([5, 0, 4, 1, 9], dtype=uint8)

In [10]:
y_train = onehot(y_train)
y_test = onehot(y_test)

In [11]:
y_train[:5]

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [12]:
mean = X_train.mean().astype(np.float32)
std_dev = X_train.std().astype(np.float32)
X_test.mean()

33.791224489795916

In [13]:
def normalize(x):
    return (x - mean)/std_dev

#### Linear Model

In [14]:
def get_linear_model():
    model = Sequential([
        Lambda(normalize,input_shape=(1,28,28)),
        Flatten(),
        Dense(10,activation='softmax')
    ])
    model.compile(optimizer=Adam(),loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [15]:
lm = get_linear_model()

In [16]:
gen = image.ImageDataGenerator()
batches = gen.flow(X_train,y_train,batch_size=batch_size)
test_batches = gen.flow(X_test,y_test,batch_size=batch_size)

In [17]:
lm.fit_generator(batches,batches.N,nb_epoch=1,
                validation_data=test_batches,nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 11s - loss: 0.4295 - acc: 0.8720 - val_loss: 0.3036 - val_acc: 0.9129


In [18]:
lm.optimizer.lr = 0.1

In [19]:
lm.fit_generator(batches,batches.N,nb_epoch=1,
                validation_data=test_batches,nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 11s - loss: 0.2984 - acc: 0.9158 - val_loss: 0.2867 - val_acc: 0.9201


In [20]:
lm.optimizer.lr = 0.01

In [21]:
lm.fit_generator(batches,batches.N,nb_epoch=3,
                validation_data=test_batches,nb_val_samples=test_batches.N)

Epoch 1/3
60000/60000 [==============================] - 11s - loss: 0.2845 - acc: 0.9199 - val_loss: 0.2911 - val_acc: 0.9194
Epoch 2/3
60000/60000 [==============================] - 11s - loss: 0.2783 - acc: 0.9217 - val_loss: 0.2756 - val_acc: 0.9242
Epoch 3/3
60000/60000 [==============================] - 11s - loss: 0.2727 - acc: 0.9241 - val_loss: 0.2772 - val_acc: 0.9237


In [22]:
lm.save_weights(model_path+'lm_0.01.h5')

In [23]:
lm.load_weights(model_path+'lm_0.01.h5')

In [79]:
lm.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 1, 28, 28)     0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 784)           0           lambda_1[0][0]                   
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 10)            7850        flatten_1[0][0]                  
Total params: 7850
____________________________________________________________________________________________________


#### Adding Dense Layers to model

In [24]:
def get_fc_model():
    model = Sequential([
        Lambda(normalize,input_shape=(1,28,28)),
        Flatten(),
        Dense(512,activation='relu'),
        Dense(10,activation='softmax')
    ])
    model.compile(optimizer=Adam(),loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [25]:
fc_model = get_fc_model()

In [26]:
fc_model.fit_generator(batches,batches.N,nb_epoch=1,
                      validation_data=test_batches,nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 12s - loss: 0.1903 - acc: 0.9419 - val_loss: 0.1055 - val_acc: 0.9668


In [27]:
fc_model.optimizer.lr = 0.1

In [28]:
fc_model.fit_generator(batches,batches.N,nb_epoch=1,
                      validation_data=test_batches,nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 12s - loss: 0.0813 - acc: 0.9746 - val_loss: 0.0728 - val_acc: 0.9784


In [29]:
fc_model.optimizer.lr = 0.01

In [30]:
fc_model.fit_generator(batches,batches.N,nb_epoch=3,
                      validation_data=test_batches,nb_val_samples=test_batches.N)

Epoch 1/3
60000/60000 [==============================] - 12s - loss: 0.0544 - acc: 0.9824 - val_loss: 0.0784 - val_acc: 0.9762
Epoch 2/3
60000/60000 [==============================] - 12s - loss: 0.0409 - acc: 0.9868 - val_loss: 0.0744 - val_acc: 0.9791
Epoch 3/3
60000/60000 [==============================] - 12s - loss: 0.0337 - acc: 0.9892 - val_loss: 0.0723 - val_acc: 0.9813


In [31]:
fc_model.save_weights(model_path+'dense_0.01.h5')

In [32]:
fc_model.load_weights(model_path+'dense_0.01.h5')

In [78]:
fc_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 1, 28, 28)     0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
flatten_2 (Flatten)              (None, 784)           0           lambda_2[0][0]                   
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 512)           401920      flatten_2[0][0]                  
____________________________________________________________________________________________________
dense_3 (Dense)                  (None, 10)            5130        dense_2[0][0]                    
Total params: 407050
______________________________________________________________________

#### Adding Convolution Layers

In [82]:
def get_conv_model_with_dense_layers():
    model = Sequential([
        Lambda(normalize,input_shape=(1,28,28)),
        ZeroPadding2D((1,1)),
        Convolution2D(32,3,3,activation='relu'),
        ZeroPadding2D((1,1)),
        Convolution2D(32,3,3,activation='relu'),
        MaxPooling2D(),
        ZeroPadding2D((1,1)),
        Convolution2D(64,3,3,activation='relu'),
        ZeroPadding2D((1,1)),
        Convolution2D(64,3,3,activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512,activation='relu'),
        Dense(10,activation='softmax'),
    ])
    model.compile(optimizer=Adam(),loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [83]:
model = get_conv_model_with_dense_layers()

In [84]:
model.fit_generator(batches,batches.N,nb_epoch=1,
                    validation_data=test_batches,nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 23s - loss: 0.1813 - acc: 0.9425 - val_loss: 0.0695 - val_acc: 0.9777


In [85]:
model.optimizer.lr = 0.1

In [86]:
model.fit_generator(batches,batches.N,nb_epoch=1,
                    validation_data=test_batches,nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 23s - loss: 0.0633 - acc: 0.9810 - val_loss: 0.0427 - val_acc: 0.9865


In [87]:
model.optimizer.lr = 0.01

In [88]:
model.fit_generator(batches,batches.N,nb_epoch=3,
                    validation_data=test_batches,nb_val_samples=test_batches.N)

Epoch 1/3
60000/60000 [==============================] - 23s - loss: 0.0480 - acc: 0.9851 - val_loss: 0.0474 - val_acc: 0.9845
Epoch 2/3
60000/60000 [==============================] - 23s - loss: 0.0419 - acc: 0.9876 - val_loss: 0.0358 - val_acc: 0.9886
Epoch 3/3
60000/60000 [==============================] - 23s - loss: 0.0377 - acc: 0.9878 - val_loss: 0.0375 - val_acc: 0.9891


In [91]:
model.save_weights(model_path+'fc_conv_0.01.h5')

In [92]:
model.load_weights(model_path+'fc_conv_0.01.h5')

In [93]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_8 (Lambda)                (None, 1, 28, 28)     0           lambda_input_8[0][0]             
____________________________________________________________________________________________________
zeropadding2d_12 (ZeroPadding2D) (None, 1, 30, 30)     0           lambda_8[0][0]                   
____________________________________________________________________________________________________
convolution2d_21 (Convolution2D) (None, 32, 28, 28)    320         zeropadding2d_12[0][0]           
____________________________________________________________________________________________________
zeropadding2d_13 (ZeroPadding2D) (None, 32, 30, 30)    0           convolution2d_21[0][0]           
___________________________________________________________________________________________

#### Augmenting Data

In [42]:
model = get_conv_model_with_dense_layers()

In [43]:
gen = image.ImageDataGenerator(rotation_range=10,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              zoom_range=0.1)

In [44]:
batches = gen.flow(X_train,y_train,batch_size=batch_size)
test_batches = gen.flow(X_test,y_test,batch_size=batch_size)

In [45]:
model.fit_generator(batches,batches.N,nb_epoch=1,
                   validation_data=test_batches,nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 19s - loss: 0.2102 - acc: 0.9324 - val_loss: 0.0922 - val_acc: 0.9733


In [46]:
model.optimizer.lr = 0.1

In [47]:
model.fit_generator(batches,batches.N,nb_epoch=1,
                   validation_data=test_batches,nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 19s - loss: 0.0698 - acc: 0.9784 - val_loss: 0.0625 - val_acc: 0.9797


In [48]:
model.optimizer.lr = 0.01

In [49]:
model.fit_generator(batches,batches.N,nb_epoch=5,
                   validation_data=test_batches,nb_val_samples=test_batches.N)

Epoch 1/5
60000/60000 [==============================] - 19s - loss: 0.0549 - acc: 0.9828 - val_loss: 0.0400 - val_acc: 0.9874
Epoch 2/5
60000/60000 [==============================] - 18s - loss: 0.0472 - acc: 0.9858 - val_loss: 0.0350 - val_acc: 0.9884
Epoch 3/5
60000/60000 [==============================] - 19s - loss: 0.0424 - acc: 0.9872 - val_loss: 0.0374 - val_acc: 0.9885
Epoch 4/5
60000/60000 [==============================] - 19s - loss: 0.0395 - acc: 0.9883 - val_loss: 0.0401 - val_acc: 0.9867
Epoch 5/5
60000/60000 [==============================] - 19s - loss: 0.0372 - acc: 0.9885 - val_loss: 0.0412 - val_acc: 0.9860


In [50]:
model.save_weights(model_path+'fc_conv_aug_0.01.h5')

In [51]:
model.load_weights(model_path+'fc_conv_aug_0.01.h5')

In [77]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_4 (Lambda)                (None, 1, 28, 28)     0           lambda_input_4[0][0]             
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 1, 30, 30)     0           lambda_4[0][0]                   
____________________________________________________________________________________________________
convolution2d_5 (Convolution2D)  (None, 32, 28, 28)    320         zeropadding2d_2[0][0]            
____________________________________________________________________________________________________
convolution2d_6 (Convolution2D)  (None, 32, 26, 26)    9248        convolution2d_5[0][0]            
___________________________________________________________________________________________

#### BatchNormalization with Augmented Data

In [52]:
def get_bn_model():
    model = Sequential([
        Lambda(normalize,input_shape=(1,28,28)),
        ZeroPadding2D((1,1)),
        Convolution2D(32,3,3,activation='relu'),
        BatchNormalization(axis=1),
        ZeroPadding2D((1,1)),
        Convolution2D(32,3,3,activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        ZeroPadding2D((1,1)),
        Convolution2D(64,3,3,activation='relu'),
        BatchNormalization(axis=1),
        ZeroPadding2D((1,1)),
        Convolution2D(64,3,3,activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Flatten(),
        Dense(512,activation='relu'),
        BatchNormalization(),
        Dense(10,activation='softmax'),
    ])
    model.compile(optimizer=Adam(),loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [53]:
bn_model = get_bn_model()

In [54]:
bn_model.fit_generator(batches,batches.N,nb_epoch=1,
                      validation_data=test_batches,nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 56s - loss: 0.1646 - acc: 0.9491 - val_loss: 0.0823 - val_acc: 0.9720


In [55]:
bn_model.optimizer.lr = 0.1

In [56]:
bn_model.fit_generator(batches,batches.N,nb_epoch=1,
                      validation_data=test_batches,nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 55s - loss: 0.0735 - acc: 0.9768 - val_loss: 0.0596 - val_acc: 0.9806


In [57]:
bn_model.optimizer.lr = 0.01

In [58]:
bn_model.fit_generator(batches,batches.N,nb_epoch=5,
                      validation_data=test_batches,nb_val_samples=test_batches.N)

Epoch 1/5
60000/60000 [==============================] - 55s - loss: 0.0562 - acc: 0.9822 - val_loss: 0.0496 - val_acc: 0.9837
Epoch 2/5
60000/60000 [==============================] - 54s - loss: 0.0480 - acc: 0.9852 - val_loss: 0.0478 - val_acc: 0.9862
Epoch 3/5
60000/60000 [==============================] - 54s - loss: 0.0431 - acc: 0.9862 - val_loss: 0.0730 - val_acc: 0.9797
Epoch 4/5
60000/60000 [==============================] - 54s - loss: 0.0427 - acc: 0.9866 - val_loss: 0.0409 - val_acc: 0.9887
Epoch 5/5
60000/60000 [==============================] - 54s - loss: 0.0400 - acc: 0.9872 - val_loss: 0.0479 - val_acc: 0.9850


In [59]:
bn_model.save_weights(model_path+'bn_aug_0.01.h5')

In [60]:
bn_model.load_weights(model_path+'bn_aug_0.01.h5')

In [76]:
bn_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_5 (Lambda)                (None, 1, 28, 28)     0           lambda_input_5[0][0]             
____________________________________________________________________________________________________
zeropadding2d_3 (ZeroPadding2D)  (None, 1, 30, 30)     0           lambda_5[0][0]                   
____________________________________________________________________________________________________
convolution2d_9 (Convolution2D)  (None, 32, 28, 28)    320         zeropadding2d_3[0][0]            
____________________________________________________________________________________________________
batchnormalization_1 (BatchNormal(None, 32, 28, 28)    64          convolution2d_9[0][0]            
___________________________________________________________________________________________

#### BatchNorm +Aug + Dropout

In [61]:
def get_bn_model_with_dropout():
    model = Sequential([
        Lambda(normalize,input_shape=(1,28,28)),
        ZeroPadding2D((1,1)),
        Convolution2D(32,3,3,activation='relu'),
        BatchNormalization(axis=1),
        ZeroPadding2D((1,1)),
        Convolution2D(32,3,3,activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        ZeroPadding2D((1,1)),
        Convolution2D(64,3,3,activation='relu'),
        BatchNormalization(axis=1),
        ZeroPadding2D((1,1)),
        Convolution2D(64,3,3,activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Flatten(),
        Dense(512,activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10,activation='softmax'),
    ])
    model.compile(optimizer=Adam(),loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [64]:
bn_model_do = get_bn_model_with_dropout()

In [65]:
bn_model_do.fit_generator(batches,batches.N,nb_epoch=1,
                      validation_data=test_batches,nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 57s - loss: 0.2197 - acc: 0.9324 - val_loss: 0.1266 - val_acc: 0.9574


In [66]:
bn_model_do.optimizer.lr = 0.1

In [68]:
bn_model_do.fit_generator(batches,batches.N,nb_epoch=1,
                      validation_data=test_batches,nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 57s - loss: 0.0913 - acc: 0.9722 - val_loss: 0.0530 - val_acc: 0.9844


In [69]:
bn_model_do.optimizer.lr = 0.01

In [70]:
bn_model_do.fit_generator(batches,batches.N,nb_epoch=5,
                      validation_data=test_batches,nb_val_samples=test_batches.N)

Epoch 1/5
60000/60000 [==============================] - 56s - loss: 0.0721 - acc: 0.9781 - val_loss: 0.0621 - val_acc: 0.9832
Epoch 2/5
60000/60000 [==============================] - 55s - loss: 0.0670 - acc: 0.9794 - val_loss: 0.0469 - val_acc: 0.9869
Epoch 3/5
60000/60000 [==============================] - 55s - loss: 0.0582 - acc: 0.9821 - val_loss: 0.0448 - val_acc: 0.9872
Epoch 4/5
60000/60000 [==============================] - 55s - loss: 0.0518 - acc: 0.9840 - val_loss: 0.0342 - val_acc: 0.9878
Epoch 5/5
60000/60000 [==============================] - 55s - loss: 0.0516 - acc: 0.9838 - val_loss: 0.0293 - val_acc: 0.9896


In [71]:
bn_model_do.optimizer.lr = 0.001

In [72]:
bn_model_do.fit_generator(batches,batches.N,nb_epoch=5,
                      validation_data=test_batches,nb_val_samples=test_batches.N)

Epoch 1/5
60000/60000 [==============================] - 56s - loss: 0.0457 - acc: 0.9867 - val_loss: 0.0255 - val_acc: 0.9922
Epoch 2/5
60000/60000 [==============================] - 55s - loss: 0.0423 - acc: 0.9868 - val_loss: 0.0348 - val_acc: 0.9881
Epoch 3/5
60000/60000 [==============================] - 55s - loss: 0.0394 - acc: 0.9875 - val_loss: 0.0363 - val_acc: 0.9907
Epoch 4/5
60000/60000 [==============================] - 55s - loss: 0.0401 - acc: 0.9878 - val_loss: 0.0387 - val_acc: 0.9887
Epoch 5/5
60000/60000 [==============================] - 55s - loss: 0.0366 - acc: 0.9887 - val_loss: 0.0963 - val_acc: 0.9858


In [73]:
bn_model_do.save_weights(model_path+'bn_aug_do_0.001.h5')

In [74]:
bn_model_do.load_weights(model_path+'bn_aug_do_0.001.h5')

In [75]:
bn_model_do.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_6 (Lambda)                (None, 1, 28, 28)     0           lambda_input_6[0][0]             
____________________________________________________________________________________________________
zeropadding2d_7 (ZeroPadding2D)  (None, 1, 30, 30)     0           lambda_6[0][0]                   
____________________________________________________________________________________________________
convolution2d_13 (Convolution2D) (None, 32, 28, 28)    320         zeropadding2d_7[0][0]            
____________________________________________________________________________________________________
batchnormalization_6 (BatchNormal(None, 32, 28, 28)    64          convolution2d_13[0][0]           
___________________________________________________________________________________________